# Implementing a File System

We will discuss how to build a Very Simple File Systyem (VSFS).

The original file system for UNIX was very much likle this.

The OS views the hard disk as a set of blocks that it can read/write from/to.

A block is 4KB large.

To the OS, the hard disk is just a set of N blocks that it can access.


<br>
<img src="images/02-blocks.png" width="500">
<br>

## How do we build a file system?

What do we need? What are we storing and how do we access it?

1) A place to store all of the user's actual data, eg. the files.
    - Files are stored across 0 or more **data blocks**.
2) Per-file metadata: creation/modication times, permissions, size in bytes, # of blocks used for the file, pointers to the blocks that hold the file. Stored in **inodes**.
    - inodes are stored in inode blocks.
3) Directories. Directories are just files that list all files and directories within them.
4) A way of tracking which inodes and data blocks are free.
    - this is done using two bitmaps, an **inodes bitmap** and a **data blocks bitmap**
5) Meta data for the entire file system itself: start/size of data region, inode region, bitmap locations, file system type, etc.
    - This is stored in the **superblock**.

<br>
<img src="images/04-blocks.png" width="500">
<br>

Whenever we want to create, read, or write to/from files, we will need to read and potentialy modify inodes, data blocks, and (sometimes) bitmaps.

<br>
<img src="images/10-inode.png" width="500">
<br>

## Example Operation: Reading from /foo/bar

Anytime we want to access any file, we always need to traverse to it from the root directory. The root directory is well-known by the OS, that is, the OS always knows its inode number.

<br>
<img src="images/05-read-foo-bar.png" width="500">
<br>

Inodes point to data blocks for that file/directory.

A data block for any directory will list the inodes of all its contents.

In order to start to read from `/foo/bar`, we have to start at the root.

1) Read in the ROOT inode
    - its inode points to its data block
2) Read in its data block
    - its data block gives us the inode for `/foo`
3) Read in `/foo`'s inode
    - in order to get `/foo`'s data block
4) Read in `/foo`'s data block
    - to get `bar`'s inode.
    - now that we have bar's inode, we have the pointers to all its data blocks and can read them in one by one.
5) Read in `bar`'s data blocks one by one, after each access update `bar`'s inode.

## Example Operation: Creating then Writing to /foo/bar

Create file bar, then write 3 block to it.

We will again have to traverse the directory structure, staring from the root and working our way to foo.

Then, to create `bar`, we need to create an inode for `bar` which means that we need to look up the inode bitmap and update it to claim a new inode.

Then to write to each block, that block must be claimed by reading and updating the data bitmap, and finally writing to the data block and updating `bar`'s inode.

<br>
<img src="images/06-write-foo-bar.png" width="500">
<br>

## Handling Large Files

A single inode can contain points to 15 data blocks, a max of referring to 60 KB for a file.

To handle large files, inodes can be chained together as a tree to expand the pointer space.